In [41]:
from io import StringIO
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import sqlite3
from datetime import datetime

# Maintaining log file

In [42]:
def log_process(message):
    with open('code_log.txt','a') as f :
        f.write(f'{datetime.now()} : {message} \n')

In [43]:
log_process('Welcome')

In [44]:
def extract(url,tableattrs):
    soup = BeautifulSoup(requests.get(url).text,'lxml')
    table = soup.find('span',string=tableattrs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_process("Data Initialization process completed , Starting Transformation process")
    return df 

In [45]:
def tranform (df,csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)

    df['MC_EUR_Billion'][4]
    
    log_process('Data transformation complete. Initiating Loading process')

    return df


In [46]:
# If you want to access a file from specific folder firstfolder then a file   (./input/)

In [47]:
def load_to_csv(df, output_path):
    """ This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing."""

    df.to_csv(output_path)

    log_process('Data saved to CSV file')

In [48]:
def load_to_db(df, sql_connection, table_name):
    """ This function saves the final data frame to a database
    table with the provided name. Function returns nothing."""

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_process('Data loaded to Database as a table, Executing queries')


def run_query(query_statement, sql_connection):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """

    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_process('Process Complete')

    return result

In [52]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    #
    log_process('Preliminaries complete. Initiating ETL process')
    #

    df = extract(url, 'By market capitalization')
    print(df)
    tranform(df, './etl_with_python/input/exchange_rate.csv')

    load_to_csv(df, output_csv_path)

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)

        print(run_query('SELECT * FROM Largest_banks', conn))

        print(run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn))

        print(run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn))

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81


FileNotFoundError: [Errno 2] No such file or directory: './etl_with_python/input/exchange_rate.csv'